# EazyML Explainable AI Template

## Define Imports

In [ ]:
!pip install --upgrade eazyml-xai
!pip install --upgrade eazyml-automl
!pip install gdown python-dotenv

In [ ]:
import os
from eazyml_xai import (
    ez_init,
    ez_explain,
    ez_get_data_type,
    create_onehot_encoded_features,
)

from eazyml import ez_display_df
import gdown
import numpy as np
import pandas as pd

from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

from dotenv import load_dotenv
load_dotenv()

## 1. Initialize EazyML
The `ez_init` function uses the `EAZYML_ACCESS_KEY` environment variable for authentication. If the variable is not set, it defaults to a trial license.

In [ ]:
ez_init(access_key=os.getenv('EAZYML_ACCESS_KEY'))

## 2. Define Dataset Files and Outcome Variable

In [ ]:
gdown.download_folder(id='1EobxYR3pg_Z3Sd4sETfe4aJLAsT98fL2')

In [ ]:
# Names of the files that will be used by EazyML APIs
train_file_path = os.path.join('data', 'Heart_Attack_traindata.csv')
test_file_path  = os.path.join('data', 'Heart_Attack_testdata.csv')

# The column name for outcome of interest
outcome = 'class'

## 3. EazyML Preprocessing Steps Before Model Training

### 3.1 Reading the Datasets and Dropping Unnecessary Columns

In [ ]:
discard_columns = []

# Reading Training Data
train = pd.read_csv(train_file_path)
train = train.drop(columns=discard_columns)

# Reading Test Data
test = pd.read_csv(test_file_path)
test = test.drop(columns=discard_columns)

### 3.2 Getting the Data Type of Features and Creating Dummy Features

In [ ]:
# Getting Data Type of Features
type_df = ez_get_data_type(train, outcome)

In [ ]:
# List of Categorical Columns
cat_list = type_df[type_df['Data Type'] == 'categorical']['Variable Name'].tolist()
cat_list = [ele for ele in cat_list if ele != outcome]

# Creating Dummy Features
train = create_onehot_encoded_features(train, cat_list)
test = create_onehot_encoded_features(test, cat_list)

## 4. Training XGBoost Classifer Model

In [ ]:
y = train[[outcome]]
X = train.drop([outcome], axis=1)

# Because y contains ['negative', 'positive']
label_encoder = LabelEncoder()
y_numeric = label_encoder.fit_transform(y)

model_name = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
       colsample_bytree=0.8, gamma=0.5, learning_rate=0.1,
       max_delta_step=0, max_depth=20, min_child_weight=1.0, missing=np.nan,
       n_estimators=10, n_jobs=1, nthread=None,
       objective='binary:logistic', random_state=0, reg_alpha=0,
       reg_lambda=5.0, scale_pos_weight=1, seed=None, silent=False,
       subsample=0.9)

model = model_name.fit(X, y_numeric)

## 5. Get Explanations

### 5.1 Get Explanations for 5 Points

In [ ]:
options = {'record_number': [1, 6, 7, 8, 9]}
response = ez_explain(train_file_path, outcome, test_file_path, model, options=options)

### 5.2 Display Explanation DataFrame

In [ ]:
ex_df = pd.DataFrame([i.values() for i in response['explanations']], columns=response['explanations'][0].keys())
ez_display_df(ex_df)